# Finnhub 뉴스 데이터 전처리 및 FinBERT 감정분석

1. 데이터 로드 및 전처리
2. FinBERT 감정분석 수행
3. 최종 결과 저장

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# 데이터 로드
df = pd.read_csv("./AAPL_extended_news_2025-06-14.csv")
print(f"원본 데이터 행 수: {len(df)}")
print(f"컬럼: {list(df.columns)}")
print(f"Summary NULL 개수: {df['summary'].isnull().sum()}")
print(f"Title NULL 개수: {df['title'].isnull().sum()}")

원본 데이터 행 수: 8769
컬럼: ['id', 'title', 'summary', 'link', 'publisher', 'category', 'pubDate', 'image', 'related', 'source', 'collection_period']
Summary NULL 개수: 166
Title NULL 개수: 0


In [2]:
# 1단계: Summary 컬럼 전처리
print("=== Summary 컬럼 전처리 ===")

# summary가 NULL인 경우 title로 대체
df['summary'] = df['summary'].fillna(df['title'])

# title과 summary 모두 NULL인 경우 제거
before_drop = len(df)
df = df.dropna(subset=['title', 'summary'], how='all')
after_drop = len(df)
print(f"title과 summary 모두 NULL인 행 제거: {before_drop - after_drop}개")

# summary 텍스트 정리 함수
def clean_text(text):
    if pd.isna(text) or text == "":
        return ""
    
    # 문자열로 변환
    text = str(text)
    
    # URL 제거
    text = re.sub(r'https?://[^\s]+', '', text)
    text = re.sub(r'www\.[^\s]+', '', text)
    
    # 줄바꿈, 탭 제거
    text = re.sub(r'[\n\t\r]', ' ', text)
    
    # 연속된 특수문자 제거 ($$$, ###, ---, === 등)
    text = re.sub(r'[#$=\-_*]{3,}', '', text)
    text = re.sub(r'[!@#$%^&*()_+=\[\]{}|;:",.<>?/~`]{5,}', '', text)
    
    # 연속된 공백을 하나로
    text = re.sub(r'\s+', ' ', text)
    
    # 앞뒤 공백 제거
    text = text.strip()
    
    return text

# summary와 title 정리
df['summary'] = df['summary'].apply(clean_text)
df['title'] = df['title'].apply(clean_text)

print("텍스트 정리 완료")
print(f"최종 데이터 행 수: {len(df)}")

=== Summary 컬럼 전처리 ===
title과 summary 모두 NULL인 행 제거: 0개
텍스트 정리 완료
최종 데이터 행 수: 8769


In [3]:
# 2단계: 날짜 처리 및 정렬
print("\n=== 날짜 처리 및 정렬 ===")

# pubDate를 datetime으로 변환
df['pubDate'] = pd.to_datetime(df['pubDate'], errors='coerce')

# 날짜 변환 실패한 행 제거
before_date_drop = len(df)
df = df.dropna(subset=['pubDate'])
after_date_drop = len(df)
print(f"날짜 변환 실패한 행 제거: {before_date_drop - after_date_drop}개")

# 날짜순 내림차순 정렬 (최신순)
df = df.sort_values('pubDate', ascending=False).reset_index(drop=True)
print(f"날짜 범위: {df['pubDate'].min()} ~ {df['pubDate'].max()}")

# full_text 생성 (title + summary)
df['full_text'] = df['title'] + ". " + df['summary']
# 빈 텍스트 제거
df = df[df['full_text'].str.strip() != ""].reset_index(drop=True)
print(f"빈 텍스트 제거 후 최종 행 수: {len(df)}")



=== 날짜 처리 및 정렬 ===
날짜 변환 실패한 행 제거: 2개
날짜 범위: 2023-03-22 18:22:51 ~ 2025-06-14 09:55:00
빈 텍스트 제거 후 최종 행 수: 8767


In [4]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 3단계: FinBERT 감정분석 (오류 수정)
print("\n=== FinBERT 감정분석 ===")

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

MODEL = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# ⭐ 해결책: truncation=True, max_length=512 추가하여 긴 텍스트 자르기
finbert = pipeline(
    "sentiment-analysis", 
    model=model, 
    tokenizer=tokenizer, 
    top_k=None,
    truncation=True,      # 텍스트 자르기 활성화 ✂️
    max_length=512,       # 최대 512 토큰으로 제한
    padding=True          # 패딩 추가
)

# 텍스트 길이 확인
texts = df['full_text'].tolist()
print(f"감정분석 대상 텍스트 수: {len(texts)}")

# 텍스트 길이 통계 확인 (처음 100개 샘플)
print("텍스트 길이 분석 중...")
sample_lengths = []
for i, text in enumerate(texts[:100]):
    tokens = tokenizer.tokenize(text)
    sample_lengths.append(len(tokens))

print(f"텍스트 길이 통계 (토큰 수, 샘플 100개):")
print(f"  평균: {sum(sample_lengths)/len(sample_lengths):.1f}")
print(f"  최대: {max(sample_lengths)}")
print(f"  최소: {min(sample_lengths)}")
print(f"  512 토큰 초과하는 텍스트: {sum(1 for x in sample_lengths if x > 512)}개")

# 감정분석 실행
print("\n🔄 감정분석 시작... (시간이 다소 걸릴 수 있습니다)")
results = finbert(texts, batch_size=4)  # 배치 크기를 줄여서 메모리 절약
print("✅ 감정분석 완료!")



=== FinBERT 감정분석 ===


C:\Users\82102\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


감정분석 대상 텍스트 수: 8767
텍스트 길이 분석 중...
텍스트 길이 통계 (토큰 수, 샘플 100개):
  평균: 66.9
  최대: 189
  최소: 19
  512 토큰 초과하는 텍스트: 0개

🔄 감정분석 시작... (시간이 다소 걸릴 수 있습니다)
✅ 감정분석 완료!


In [6]:
# 4단계: 감정분석 결과 처리 및 최종 데이터프레임 구성
print("\n=== 감정분석 결과 처리 ===")

# 감정분석 점수 추출
pos_scores, neu_scores, neg_scores = [], [], []
sentiment_labels = []

for res in results:
    # 각 감정의 점수를 딕셔너리로 정리
    d = {r["label"].lower(): r["score"] for r in res}
    pos_score = d.get("positive", 0.0)
    neu_score = d.get("neutral", 0.0)
    neg_score = d.get("negative", 0.0)
    
    pos_scores.append(pos_score)
    neu_scores.append(neu_score)
    neg_scores.append(neg_score)
    
    # 가장 높은 점수의 감정을 주 감정으로 결정
    max_score = max(pos_score, neu_score, neg_score)
    if max_score == pos_score:
        sentiment_labels.append("positive")
    elif max_score == neg_score:
        sentiment_labels.append("negative")
    else:
        sentiment_labels.append("neutral")

# 결과를 데이터프레임에 추가
df['pos'] = pos_scores
df['neu'] = neu_scores
df['neg'] = neg_scores
df['sentiment'] = sentiment_labels

print(f"감정분석 결과:")
print(f"- Positive: {sentiment_labels.count('positive')}개")
print(f"- Neutral: {sentiment_labels.count('neutral')}개") 
print(f"- Negative: {sentiment_labels.count('negative')}개")



=== 감정분석 결과 처리 ===
감정분석 결과:
- Positive: 2384개
- Neutral: 4731개
- Negative: 1652개


In [7]:
# 5단계: 최종 결과 저장
print("\n=== 최종 결과 저장 ===")

# 요구사항에 맞는 컬럼만 선택하여 최종 데이터프레임 생성
final_df = pd.DataFrame({
    'Date': df['pubDate'].dt.strftime('%Y-%m-%d %H:%M:%S'),  # 날짜 포맷 지정
    'full_text': df['full_text'],
    'sentiment': df['sentiment'],  # positive/negative/neutral 중 하나
    'neg': df['neg'],
    'neu': df['neu'], 
    'pos': df['pos']
})

# 최종 결과 저장
output_filename = "AAPL_finnhub_processed_final.csv"
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"✅ 최종 전처리된 데이터 저장 완료: {output_filename}")
print(f"📊 최종 데이터 행 수: {len(final_df)}")
print(f"📋 최종 컬럼: {list(final_df.columns)}")
print(f"\n감정분석 결과 분포:")
print(final_df['sentiment'].value_counts())

# 샘플 데이터 확인
print(f"\n처음 3개 샘플:")
print(final_df[['Date', 'sentiment', 'pos', 'neu', 'neg']].head(3))



=== 최종 결과 저장 ===
✅ 최종 전처리된 데이터 저장 완료: AAPL_finnhub_processed_final.csv
📊 최종 데이터 행 수: 8767
📋 최종 컬럼: ['Date', 'full_text', 'sentiment', 'neg', 'neu', 'pos']

감정분석 결과 분포:
sentiment
neutral     4731
positive    2384
negative    1652
Name: count, dtype: int64

처음 3개 샘플:
                  Date sentiment       pos       neu       neg
0  2025-06-14 09:55:00  negative  0.000066  0.000041  0.999893
1  2025-06-14 01:30:33   neutral  0.027770  0.944107  0.028124
2  2025-06-14 01:00:00  negative  0.000960  0.456238  0.542802
